# SCAN Pipeline: SimCLR + Clustering on LULC Dataset
This notebook implements Step 1 (SimCLR pretext training) and Step 2 (SCAN clustering) for unsupervised learning on a custom Land Use/Land Cover (LULC) image dataset.

In [17]:
import os
import torch
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.neighbors import NearestNeighbors

from simclr_utils import SimCLRDataset, train_simclr
from simclr_loss import NTXentLoss
from scan_model import get_backbone, ClusterHead
from train_scan import train_scan

## 📁 Step 0: Load Flat Folder LULC Dataset

In [18]:
class FlatFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = glob(os.path.join(folder_path, '*'))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            return self.transform(img)
        return img

In [19]:
# Transformations
transform_simclr = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor()
])

base_transform = transforms.Compose([
    transforms.Resize((64, 64)),  # ✅ Ensures all images are exactly [3, 64, 64]
    transforms.ToTensor()
])

# Dataset path
dataset_path = "eu_jpg"
batch_size = 64

In [20]:
# Load datasets
flat_dataset = FlatFolderDataset(dataset_path, transform=None)
simclr_dataset = SimCLRDataset(flat_dataset, transform_simclr)
base_dataset = FlatFolderDataset(dataset_path, transform=base_transform)

# Data loaders
simclr_loader = DataLoader(simclr_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
base_loader   = DataLoader(base_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

## 🏋️‍♂️ Step 1: Train SimCLR Pretext Model

In [21]:
feature_dim = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

backbone = get_backbone(name='resnet18', dim=feature_dim).to(device)
criterion = NTXentLoss(temperature=0.5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = NTXentLoss(temperature=0.5, device=device)

train_simclr(backbone, simclr_loader, criterion, epochs=100, device=device)
torch.save(backbone.state_dict(), "simclr_lulc_resnet18_epoch100_pretrained.pth")


c:\Users\it08d\.conda\envs\scan\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\it08d\.conda\envs\scan\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/100:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (89740975 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 1/100:  50%|█████     | 1/2 [00:20<00:20, 20.60s/it]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (134843184 pixels) exceeds limit of 894784

Epoch [1/100] Loss: 499999904.0000


Epoch 2/100:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (109820772 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 2/100:  50%|█████     | 1/2 [00:21<00:21, 21.93s/it]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (93442472 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 2/100: 100%|██████████| 2/2 [00:39<00:00, 19.90s/it]


Epoch [2/100] Loss: 499999904.0000


Epoch 3/100:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (115557800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 3/100: 100%|██████████| 2/2 [00:35<00:00, 17.73s/it]


...

Epoch [98/100] Loss: 499999904.0000


Epoch 99/100:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (104486760 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 99/100: 100%|██████████| 2/2 [00:45<00:00, 22.65s/it]


Epoch [99/100] Loss: 499999904.0000


Epoch 100/100:   0%|          | 0/2 [00:00<?, ?it/s]c:\Users\it08d\.conda\envs\scan\Lib\site-packages\PIL\Image.py:3452: DecompressionBombWarning: Image size (91091448 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Epoch 100/100: 100%|██████████| 2/2 [00:51<00:00, 25.50s/it]


Epoch [100/100] Loss: 499999904.0000


## 🔍 Step 2: Extract Features and Build k-NN Graph

In [22]:
# Extract features
backbone.eval()
features = []

with torch.no_grad():
    for x in tqdm(base_loader):
        x = x.to(device)
        z = backbone(x)
        z = torch.nn.functional.normalize(z, dim=1)
        features.append(z.cpu())

features = torch.cat(features, dim=0)
np.save("simclr_lulc_resnet18_epoch100_feature.npy", features.numpy())

100%|██████████| 2/2 [00:45<00:00, 22.73s/it]


## 🔗 Step 2: Train SCAN Clustering Head

In [23]:
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import torch

from train_scan import train_scan
from scan_model import ClusterHead

# Load precomputed features
features = np.load("simclr_lulc_resnet18_epoch100_feature.npy")
features_tensor = torch.tensor(features, dtype=torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Build neighbors once (before the loop)
N = features.shape[0]
k = min(5, N - 1)
neighbors = NearestNeighbors(n_neighbors=k + 1, algorithm='auto').fit(features)
_, indices = neighbors.kneighbors(features)
neighbors_indices = [inds[1:] for inds in indices]  # Exclude self

# Store results
results = []

for n_clusters in range(5, 11):
    print(f"\n🔁 Training SCAN for n_clusters = {n_clusters}")

    # Build new cluster head
    cluster_head = ClusterHead(n_clusters=n_clusters, feature_dim=features.shape[1]).to(device)

    # Train SCAN
    train_scan(features_tensor, neighbors_indices, cluster_head, epochs=50, device=device)

    # Get predictions
    with torch.no_grad():
        preds = cluster_head(features_tensor.to(device)).cpu()
        labels = preds.argmax(dim=1).numpy()

    # Internal validation metrics
    sil_score = silhouette_score(features, labels)
    db_score = davies_bouldin_score(features, labels)
    ch_score = calinski_harabasz_score(features, labels)

    results.append({
        'n_clusters': n_clusters,
        'avg_silhouette_score': sil_score,
        'avg_davies_bouldin_index': db_score,
        'avg_calinski_harabasz_score': ch_score
    })

# Save results
results_df = pd.DataFrame(results)
output_path = "[Testcase 18]_SCAN.csv"
results_df.to_csv(output_path, index=False)

print(f"\n✅ SCAN clustering evaluation complete. Results saved to:\n{output_path}")


🔁 Training SCAN for n_clusters = 5


Epoch 1/50: 100%|██████████| 1/1 [00:00<00:00,  6.78it/s]


[SCAN] Epoch 1: Loss = -6.4344


Epoch 2/50: 100%|██████████| 1/1 [00:00<00:00, 10.21it/s]


[SCAN] Epoch 2: Loss = -6.4350


...

[SCAN] Epoch 48: Loss = -9.2448


Epoch 49/50: 100%|██████████| 1/1 [00:00<00:00,  6.36it/s]


[SCAN] Epoch 49: Loss = -9.2465


Epoch 50/50: 100%|██████████| 1/1 [00:00<00:00,  4.18it/s]

[SCAN] Epoch 50: Loss = -9.2481

✅ SCAN clustering evaluation complete. Results saved to:
[Testcase 18]_SCAN.csv
